In [ ]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["savefig.dpi"] = 300
rcParams["figure.dpi"] = 300

from celerite import plot_setup
plot_setup.setup(auto=False)

In [ ]:
import kplr
import copy
import emcee3
import pickle
import fitsio
import corner
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

import transit

from celerite.plot_setup import setup, get_figsize, COLORS

import celerite
from celerite import terms, modeling

In [ ]:
class RotationTerm(terms.Term):
    parameter_names = ("log_amp", "log_timescale", "log_period", "log_factor")

    def get_real_coefficients(self, params):
        log_amp, log_timescale, log_period, log_factor = params
        f = np.exp(log_factor)
        return (
            np.exp(log_amp) * (1.0 + f) / (2.0 + f),
            np.exp(-log_timescale),
        )

    def get_complex_coefficients(self, params):
        log_amp, log_timescale, log_period, log_factor = params
        f = np.exp(log_factor)
        return (
            np.exp(log_amp) / (2.0 + f),
            0.0,
            np.exp(-log_timescale),
            2*np.pi*np.exp(-log_period),
        )

class TransitModel(modeling.Model):
    parameter_names = ("mean_flux", "log_period", "log_ror", "log_duration",
                       "t0", "impact", "q1", "q2")

    def __init__(self, texp, *args, **kwargs):
        self.texp = texp
        super(TransitModel, self).__init__(*args, **kwargs)

    def get_value(self, t):
        system = transit.SimpleSystem(
            period=np.exp(self.log_period),
            ror=np.exp(self.log_ror),
            duration=np.exp(self.log_duration),
            t0=self.t0,
            impact=self.impact,
            q1=self.q1,
            q2=self.q2
        )
        lc = system.light_curve(t, texp=self.texp)
        return 1e3 * (lc - 1.0) + self.mean_flux

In [ ]:
setup(auto=True)
np.random.seed(42)

data = fitsio.read("../data/kplr001430163-2013011073258_llc.fits")
texp = 1625.3467838829 / 60. / 60. / 24.

N = 1000
m = data["SAP_QUALITY"] == 0
m &= np.isfinite(data["TIME"])
m &= np.isfinite(data["PDCSAP_FLUX"])
t = np.ascontiguousarray(data["TIME"][m], dtype=np.float64)[:N]
y = np.ascontiguousarray(data["PDCSAP_FLUX"][m], dtype=np.float64)[:N]
yerr = np.ascontiguousarray(data["PDCSAP_FLUX_ERR"][m], dtype=np.float64)[:N]
t -= 0.5 * (t.min() + t.max())

# Build the true model
true_model = TransitModel(
    texp,
    0.0,
    np.log(8.0),    # period
    np.log(0.015),  # Rp / Rs
    np.log(0.5),    # duration
    0.0,            # t_0
    0.5,            # impact
    0.5,            # q_1
    0.5,            # q_2
)
true_params = np.array(true_model.get_parameter_vector())

# Inject the transit into the data
true_transit = 1e-3*true_model.get_value(t) + 1.0
y *= true_transit

# Normalize the data
med = np.median(y)
y = (y / med - 1.0) * 1e3
yerr *= 1e3 / med

# Set up the GP model
mean = TransitModel(
    texp,
    0.0,
    np.log(8.0),
    np.log(0.015),
    np.log(0.5),
    0.0,
    0.5,
    0.5,
    0.5,
    bounds=[
        (-0.5, 0.5),
        np.log([7.9, 8.1]),
        (np.log(0.005), np.log(0.1)),
        (np.log(0.4), np.log(0.6)),
        (-0.1, 0.1),
        (0, 1.0), (0, 1), (0, 1)
    ]
)

kernel = RotationTerm(
    np.log(np.var(y)), np.log(0.5*t.max()), np.log(4.5), 0.0,
    bounds=dict(
        log_amp=(-10.0, 0.0),
        log_timescale=(1.5, 5.0),
        log_period=(-3.0, 5.0),
        log_factor=(-5.0, 5.0),
    ),
)
kernel += terms.JitterTerm(
    log_sigma=np.log(0.5*yerr.min()),
    bounds=[(-5.0, 0.0)],
)

gp = celerite.GP(kernel, mean=mean, fit_mean=True)
gp.compute(t, yerr)
print("Initial log-likelihood: {0}".format(gp.log_likelihood(y)))

In [ ]:
def neg_log_like(params, y, gp):
    gp.set_parameter_vector(params)
    return -gp.log_likelihood(y)

# Optimize with random restarts
p0 = gp.get_parameter_vector()
bounds = gp.get_parameter_bounds()
r = minimize(neg_log_like, p0, method="L-BFGS-B", bounds=bounds, args=(y, gp))
gp.set_parameter_vector(r.x)
ml_params = np.array(r.x)
print("Maximum log-likelihood: {0}".format(gp.log_likelihood(y)))

# Compute the maximum likelihood predictions
x = np.linspace(t.min(), t.max(), 5000)
trend = gp.predict(y, t, return_cov=False)
trend -= gp.mean.get_value(t) - gp.mean.mean_flux
mu, var = gp.predict(y, x, return_var=True)
std = np.sqrt(var)
mean_mu = gp.mean.get_value(x)
mu -= mean_mu
wn = np.exp(2*gp.kernel.terms[1].log_sigma)
ml_yerr = np.sqrt(yerr**2 + wn)

# Plot the maximum likelihood predictions
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=get_figsize(1, 2))
ax1.errorbar(t - t.min(), y, yerr=ml_yerr, fmt=".k", capsize=0, zorder=-1)
ax1.plot(x - t.min(), mu, zorder=100)
ax1.set_ylim(-0.72, 0.72)
ax1.yaxis.set_major_locator(plt.MaxNLocator(5))
ax1.set_ylabel("raw [ppt]")
ax1.yaxis.set_label_coords(-0.1, 0.5)

ax2.errorbar(t - t.min(), y-trend, yerr=ml_yerr, fmt=".k", capsize=0,
             zorder=-1)
ax2.plot(x - t.min(), mean_mu - gp.mean.mean_flux, zorder=100)
ax2.set_xlim(0, t.max()-t.min())
ax2.set_ylim(-0.41, 0.1)

ax2.annotate("N = {0}".format(len(t)), xy=(0, 0),
             xycoords="axes fraction",
             xytext=(5, 5), textcoords="offset points",
             ha="left", va="bottom")

ax2.yaxis.set_major_locator(plt.MaxNLocator(5))
ax2.set_ylabel("de-trended [ppt]")
ax2.set_xlabel("time [days]")
ax2.yaxis.set_label_coords(-0.1, 0.5)
fig.savefig("transit-ml.pdf")

In [ ]:
with open("transit.pkl", "wb") as f:
    pickle.dump((gp, y, true_model.get_parameter_dict()), f, -1)

# Do the MCMC
def log_prob(params):
    gp.set_parameter_vector(params)
    lp = gp.log_prior()
    if not np.isfinite(lp):
        return -np.inf
    return gp.log_likelihood(y) + lp

# Initialize
print("Running MCMC sampling...")
ndim = len(ml_params)
nwalkers = 32
pos = ml_params + 1e-5 * np.random.randn(nwalkers, ndim)
lp = np.array(list(map(log_prob, pos)))
m = ~np.isfinite(lp)
while np.any(m):
    pos[m] = ml_params + 1e-5 * np.random.randn(m.sum(), ndim)
    lp[m] = np.array(list(map(log_prob, pos[m])))
    m = ~np.isfinite(lp)

# Sample
sampler = emcee3.Sampler(backend=emcee3.backends.HDFBackend("transit.h5"))
with emcee3.pools.InterruptiblePool() as pool:
    ensemble = emcee3.Ensemble(emcee3.SimpleModel(log_prob), pos, pool=pool)
    sampler.run(ensemble, 30000, progress=True)

In [ ]:
true_params = true_model.get_parameter_dict()
names = gp.get_parameter_names()
cols = ["log_period", "log_ror", "log_duration", "t0"]
inds = [names.index("mean:{0}".format(c)) for c in cols]
samples = np.array(sampler.get_coords(discard=10000, flat=True, thin=7))
samples = samples[:, inds]
samples[:, :-1] = np.exp(samples[:, :-1])
truths = np.array([true_params[k] for k in cols])
truths[:-1] = np.exp(truths[:-1])
fig = corner.corner(samples, truths=truths, smooth=0.5,
                    labels=[r"period", r"$R_\mathrm{P}/R_\star$", r"duration",
                            r"$t_0$"])
for ax in np.array(fig.axes).flatten():
    ax.xaxis.set_label_coords(0.5, -0.4)
    ax.yaxis.set_label_coords(-0.4, 0.5)
fig.savefig("transit-corner.pdf")

In [ ]:
plt.plot(sampler.get_coords()[:, :, names.index("mean:impact")]);

In [ ]:
from scipy.linalg import cho_solve, cho_factor

p0 = gp.get_parameter_vector()
fast_timing = %timeit -o log_prob(p0)

def _time_this():
    K = gp.get_matrix(include_diagonal=True)
    factor = cho_factor(K, overwrite_a=True)
    ld = 2.0 * np.sum(np.log(np.diag(factor[0])))
    resid = gp.mean.get_value(t) - y
    ll = -0.5*(np.dot(resid, cho_solve(factor, resid))+ld) + gp.log_prior()

slow_timing = %timeit -o _time_this()

In [ ]:
samples = sampler.get_coords(discard=10000, flat=True)
names = gp.get_parameter_names()
chain = sampler.get_coords(discard=10000)[:, :, names.index("mean:log_period")]
tau = np.mean(emcee3.autocorr.integrated_time(np.mean(chain, axis=1), c=5))
neff = len(samples) / tau
tau, neff

In [ ]:
import json
c = gp.kernel.coefficients
with open("transit.json", "w") as f:
    json.dump(dict(
        N=len(t),
        J=len(c[0]) + len(c[2]),
        tau=tau,
        neff=neff,
        time=fast_timing.average,
        direct_time=slow_timing.average,
        nwalkers=nwalkers,
        nburn=10000,
        nsteps=30000,
        ndim=ndim,
    ), f)

In [ ]:
name_map = {
    'kernel:terms[0]:log_amp': "$\ln(B/\mathrm{ppt}^2)$",
    'kernel:terms[0]:log_timescale': "$\ln(L/\mathrm{day})$",
    'kernel:terms[0]:log_period': "$\ln(P_\mathrm{rot}/\mathrm{day})$",
    'kernel:terms[0]:log_factor': "$\ln(C)$",
    'kernel:terms[1]:log_sigma': "$\ln(\sigma/\mathrm{ppt})$",
    'mean:mean_flux': "$f_0/\mathrm{ppt}$",
    'mean:log_period': "$\ln(P_\mathrm{orb}/\mathrm{day})$",
    'mean:log_ror': "$\ln(R_p/R_\star)$",
    'mean:log_duration': "$\ln(T/\mathrm{day})$",
    'mean:t0': "$t_0/\mathrm{day}$",
    'mean:impact': "$b$",
    'mean:q1': "$q_1$",
    'mean:q2': "$q_2$",
}
params = list(zip(
    (name_map[n] for n in gp.get_parameter_names()),
    gp.get_parameter_bounds()
))

params[6] = "$\ln(P_\mathrm{orb}/\mathrm{day})$", ["\ln(7.9)", "\ln(8.1)"]
params[7] = "$\ln(R_p/R_\star)$", ["\ln(0.005)", "\ln(0.1)"]
params[8] = "$\ln(T/\mathrm{day})$", ["\ln(0.4)", "\ln(0.6)"]
with open("transit-params.json", "w") as f:
    json.dump(params, f)